In [1]:
import numpy as np
import pandas as pd
from tqdm import tqdm
import gc
import os
from glob import glob
from multiprocessing import Pool
nthread = 16
from time import time
import lightgbm as lgb
import utils

In [2]:
param = {
         'objective': 'binary',
         'metric': 'auc',
         'learning_rate': 0.2,
         'max_depth': 4,
         'num_leaves': 2**4-1,
         'max_bin': 100,
         'min_child_samples': 300,
         'min_child_weight': 0,
         'colsample_bytree': 0.8,
         'subsample': 0.1,
         'nthread': 64,
         'scale_pos_weight': 100,}

evals_result = {}

In [3]:
train = pd.read_pickle('../data/train/080.p')
valid = pd.read_pickle('../data/train/099.p')
gc.collect()

0

In [4]:
dtrain1 = lgb.Dataset(train[['ip', 'app', 'device', 'os', 'channel']],
            label=train.is_attributed,
            #feature_name=['ip', 'app', 'device', 'os', 'channel'],
            categorical_feature=['ip', 'app', 'device', 'os', 'channel', 'unko']
           )
#dtrain1.save_binary('../data/tmp_train.mt')

In [5]:
dvalid1 = lgb.Dataset(valid[['ip', 'app', 'device', 'os', 'channel']],
            label=valid.is_attributed,
            #feature_name=['ip', 'app', 'device', 'os', 'channel'],
            categorical_feature=['ip', 'app', 'device', 'os', 'channel']
           )
#dvalid1.save_binary('../data/tmp_valid.mt')

In [66]:
dtrain2 = lgb.Dataset('../data/tmp_train.mt')
dvalid2 = lgb.Dataset('../data/tmp_valid.mt')

In [68]:
dtrain2.set_feature_name(['ip', 'app', 'device', 'os', 'channel'])
dvalid2.set_feature_name(['ip', 'app', 'device', 'os', 'channel'])

In [64]:
dtrain2.set_categorical_feature(['ip', 'app', 'device', 'os', 'channel'])
dvalid2.set_categorical_feature(['ip', 'app', 'device', 'os', 'channel'])

In [6]:
model = lgb.train(params=param, train_set=dtrain1, num_boost_round=50, 
                      valid_sets=[dtrain1, dvalid1], 
                      valid_names=['train','valid'], 
                      early_stopping_rounds=50, 
                      evals_result=evals_result, 
                      verbose_eval=10,
                     categorical_feature=['ip', 'app', 'device', 'os', 'channel']
                      )

Training until validation scores don't improve for 50 rounds.


KeyboardInterrupt: 

In [78]:
dtrain1.get_field('ip')

LightGBMError: b'Field not found'

In [77]:
dtrain1.construct()

In [3]:
test =   utils.read_pickles('../data/test', ['ip', 'app', 'device', 'os', 'channel', 'click_time'])
gc.collect()

100%|██████████| 100/100 [00:10<00:00,  9.80it/s]


232

In [4]:
def f1():
    ip, app, device, os, channel, click_time, attributed_time, is_attributed = train.sample(1).values[0]
    tmp = train[train.ip==ip]
    tmp = tmp[tmp.app==app][tmp.device==device][tmp.os==os][tmp.channel==channel]
    gc.collect()
    return tmp

In [5]:
f1()

,ip,app,device,os,channel,click_time,attributed_time,is_attributed
11139133,203847,3,1,19,280,2017-11-07 00:30:38,NaT,0
11192183,203847,3,1,19,280,2017-11-07 00:31:28,NaT,0
15088343,203847,3,1,19,280,2017-11-07 01:38:46,NaT,0


In [12]:
f1()

,ip,app,device,os,channel,click_time,attributed_time,is_attributed
9560868,34385,9,1,48,334,2017-11-07 00:04:28,NaT,0
10006847,34385,9,1,48,334,2017-11-07 00:12:10,NaT,0


In [5]:
tmp = train[train.ip==5314]
tmp = tmp[tmp.app==72][tmp.device==1][tmp.os==13][tmp.channel==101]

In [6]:
tmp

,ip,app,device,os,channel,click_time,attributed_time,is_attributed
12669743,5314,72,1,13,101,2017-11-07 00:55:41,2017-11-07 06:45:56,1
17060606,5314,72,1,13,101,2017-11-07 02:15:54,2017-11-07 03:08:38,1
37469385,5314,72,1,13,101,2017-11-07 08:47:37,2017-11-07 09:09:14,1
43745916,5314,72,1,13,101,2017-11-07 10:50:08,2017-11-07 11:37:13,1
49595237,5314,72,1,13,101,2017-11-07 12:42:50,NaT,0
50375332,5314,72,1,13,101,2017-11-07 12:58:32,2017-11-07 13:09:06,1
51673683,5314,72,1,13,101,2017-11-07 13:23:07,2017-11-07 13:45:22,1
62935431,5314,72,1,13,101,2017-11-07 17:38:59,2017-11-07 23:06:31,1
94152223,5314,72,1,13,101,2017-11-08 07:42:02,2017-11-08 09:01:06,1
111576903,5314,72,1,13,101,2017-11-08 13:02:00,2017-11-08 13:03:54,1


In [8]:
tmp = test[test.ip==5314]
tmp = tmp[tmp.app==72][tmp.device==1][tmp.os==13][tmp.channel==101]
tmp

,ip,app,device,os,channel,click_time
2007040,5314,72,1,13,101,2017-11-10 04:35:25
2904166,5314,72,1,13,101,2017-11-10 04:51:42
2906737,5314,72,1,13,101,2017-11-10 04:51:45
18153445,5314,72,1,13,101,2017-11-10 14:47:34


In [6]:
keys = ['ip', 'app', 'device', 'os', 'channel']

In [7]:

gr = train.groupby(keys)
sum_ = gr['is_attributed'].sum()
size = gr.size()
df = pd.concat([sum_, size], axis=1)
df.columns = ['a', 'b']
df[c] = df.a/df.b
    

NameError: name 'c' is not defined

In [8]:
df['c'] = df.a/df.b

In [9]:
df

a  b     c
ip     app device os channel            
1      1   1      2  125      0  1  0.00
                  9  125      0  1  0.00
                  48 13       0  1  0.00
       2   1      2  477      0  5  0.00
                  6  477      0  1  0.00
                  9  477      0  1  0.00
                  15 477      0  1  0.00
                  48 477      0  1  0.00
                  49 477      0  1  0.00
                  52 477      0  1  0.00
       3   1      48 182      0  1  0.00
                  52 182      0  1  0.00
       4   1      48 101      0  1  0.00
                  52 101      0  1  0.00
       5   1      4  113      1  1  1.00
                  27 113      1  1  1.00
                  30 113      1  1  1.00
                  32 113      1  1  1.00
                  37 113      1  1  1.00
       6   1      15 459      0  1  0.00
       7   1      48 101      0  2  0.00
                  52 101      0  2  0.00
       8   1      9  259      0  1  0.00
       9   1      3  134      0  1  0.00
                  9  258      0  1  0.00
                     322      0  1  0.00
                  10 134      0  1  0.00
       10  1      4  113      1  1  1.00
                  30 113      1  1  1.00
                  43 113      1  1  1.00
...                          .. ..   ...
364776 24  2      58 105      0  2  0.00
       25  2      13 259      0  4  0.00
       26  1      13 266      0  1  0.00
       27  1      19 122      0  1  0.00
       34  1      19 243      1  2  0.50
       47  2      13 484      0  1  0.00
       55  2      13 406      0  1  0.00
       56  1      13 406      0  1  0.00
       64  1      13 459      0  1  0.00
       93  1      13 371      0  1  0.00
       94  1      13 361      0  1  0.00
       160 2      58 457      0  1  0.00
364777 3   1      19 280      0  1  0.00
       19  0      21 213      1  4  0.25
364778 1   1      13 17       0  1  0.00
       2   1      13 377      0  1  0.00
       3   1      13 489      0  1  0.00
       12  1      13 205      0  1  0.00
       13  1      13 477      0  1  0.00
       14  1      13 401      0  1  0.00
       15  1      13 412      0  1  0.00
       18  1      13 30       0  1  0.00
                     134      0  1  0.00
       22  1      13 116      0  1  0.00
       23  1      13 30       0  1  0.00
       26  1      13 477      0  1  0.00
       29  1      18 101      1  1  1.00
       56  1      13 406      0  1  0.00
       93  1      13 371      0  1  0.00
       94  1      13 361      0  1  0.00

[53857276 rows x 3 columns]